# Multi-task learning Demo

For demo purpose, we build our toy datasets since it is simpler to train and visualize the result.

Here we randomly generate 100,000 data points in 2D space. Each axis is in the range from 0 to 1.

For the output Y, we have 3 labels in the following logic

![jupyter](https://gitcdn.xyz/cdn/Tony607/blog_statics/eb71596f3a3dd9f3d78bbc2691e8326706567aaf/images/multitask/logics.png)

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import random

N =100000
X = np.random.rand(N,2)
# place holder for Y
Y = np.ones((N,3))
Y[:,0] = X[:, 1] <= 0.5
Y[:,1] = X[:, 0] >= 0.5
Y[:,2] = X[:, 0] + X[:, 1] > 1

In [6]:
print(X[:, 1][-1])

0.507707728873725


In [3]:
print(X[:, 1] <= 0.5)

[False False False ...  True False False]


In [4]:
print(X[:, 1][-1] <= 0.5)

False


We will build a model to discover such relation between X and Y,

To make the problem more complicated, we will simulate the labeler to drop some of the output labels.

In [10]:
# Mask for missing label.
mask_value = -1
# Drop 2% y0.
Y[: int(N*0.020), 0] = mask_value
# Drop 0.7% y1.
Y[int(N*0.0018): int(N*0.0025), 1] = mask_value
# Drop 1.1% y2.
Y[int(N*0.0024): int(N*0.0035), 2] = mask_value

In [11]:
print(Y[: int(N*0.020), 0])

[-1. -1. -1. ... -1. -1. -1.]


In [12]:
print(Y[: int(N*0.018), 0][-1])

-1.0


In [11]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(10, input_dim=2, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


Here is the important part, where we define our custom loss function to "mask" only labeled data.

The mask will be a tensor to store 3 values for each training sample whether the label is not equal to our mask_value (-1),

Then during computing the binary cross-entropy loss, we only compute those masked losses

In [12]:
from keras import backend as K
def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
    return K.binary_crossentropy(y_true * mask, y_pred * mask)

model.compile(loss=masked_loss_function, optimizer='adam', metrics=['accuracy'])

Training is simple, let's first reserve the last 3000 generated data for the final evaluation test.

Then split the rest data into 90% for train and 10% for dev during training.

In [13]:
# split into 90% for train and 10% for dev
X_train, X_dev, y_train, y_dev = train_test_split(X[:-3000], Y[:-3000], test_size=0.9, 
                                                  random_state=random.seed(9001))

history = model.fit(X_train, y_train, validation_data=(X_dev,y_dev), 
                    epochs=2000, batch_size=5000)

Instructions for updating:
Use tf.cast instead.
Train on 9700 samples, validate on 87300 samples
Epoch 1/2000
9700/9700 [==============================] - 1s 57us/step - loss: 0.6858 - acc: 0.2640 - val_loss: 0.6840 - val_acc: 0.2718
Epoch 2/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.6835 - acc: 0.2732 - val_loss: 0.6816 - val_acc: 0.2854
Epoch 3/2000
9700/9700 [==============================] - 0s 6us/step - loss: 0.6810 - acc: 0.2832 - val_loss: 0.6792 - val_acc: 0.2946
Epoch 4/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.6785 - acc: 0.2904 - val_loss: 0.6768 - val_acc: 0.3009
Epoch 5/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.6761 - acc: 0.2959 - val_loss: 0.6746 - val_acc: 0.3046
Epoch 6/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.6738 - acc: 0.3368 - val_loss: 0.6724 - val_acc: 0.3529
Epoch 7/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.6716 -

Epoch 118/2000
9700/9700 [==============================] - 0s 9us/step - loss: 0.0928 - acc: 0.5919 - val_loss: 0.0920 - val_acc: 0.5920
Epoch 119/2000
9700/9700 [==============================] - 0s 8us/step - loss: 0.0917 - acc: 0.5926 - val_loss: 0.0909 - val_acc: 0.5920
Epoch 120/2000
9700/9700 [==============================] - 0s 6us/step - loss: 0.0906 - acc: 0.5924 - val_loss: 0.0898 - val_acc: 0.5910
Epoch 121/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0895 - acc: 0.5911 - val_loss: 0.0887 - val_acc: 0.5906
Epoch 122/2000
9700/9700 [==============================] - 0s 6us/step - loss: 0.0885 - acc: 0.5907 - val_loss: 0.0877 - val_acc: 0.5905
Epoch 123/2000
9700/9700 [==============================] - 0s 6us/step - loss: 0.0875 - acc: 0.5913 - val_loss: 0.0867 - val_acc: 0.5910
Epoch 124/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0865 - acc: 0.5912 - val_loss: 0.0858 - val_acc: 0.5905
Epoch 125/2000
9700/9700 [========

9700/9700 [==============================] - 0s 5us/step - loss: 0.0390 - acc: 0.6204 - val_loss: 0.0391 - val_acc: 0.6210
Epoch 237/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0388 - acc: 0.6211 - val_loss: 0.0389 - val_acc: 0.6227
Epoch 238/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0386 - acc: 0.6251 - val_loss: 0.0387 - val_acc: 0.6242
Epoch 239/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0384 - acc: 0.6241 - val_loss: 0.0385 - val_acc: 0.6244
Epoch 240/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0382 - acc: 0.6248 - val_loss: 0.0383 - val_acc: 0.6238
Epoch 241/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0381 - acc: 0.6254 - val_loss: 0.0382 - val_acc: 0.6248
Epoch 242/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0379 - acc: 0.6254 - val_loss: 0.0380 - val_acc: 0.6262
Epoch 243/2000
9700/9700 [=======================

9700/9700 [==============================] - 0s 5us/step - loss: 0.0254 - acc: 0.6977 - val_loss: 0.0255 - val_acc: 0.6980
Epoch 355/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0253 - acc: 0.6979 - val_loss: 0.0255 - val_acc: 0.6979
Epoch 356/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0252 - acc: 0.6978 - val_loss: 0.0254 - val_acc: 0.6988
Epoch 357/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0252 - acc: 0.6980 - val_loss: 0.0253 - val_acc: 0.7002
Epoch 358/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0251 - acc: 0.7001 - val_loss: 0.0253 - val_acc: 0.7001
Epoch 359/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0250 - acc: 0.6990 - val_loss: 0.0252 - val_acc: 0.7004
Epoch 360/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0249 - acc: 0.7011 - val_loss: 0.0251 - val_acc: 0.7020
Epoch 361/2000
9700/9700 [=======================

9700/9700 [==============================] - 0s 5us/step - loss: 0.0192 - acc: 0.7534 - val_loss: 0.0193 - val_acc: 0.7531
Epoch 473/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0191 - acc: 0.7521 - val_loss: 0.0192 - val_acc: 0.7523
Epoch 474/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0192 - acc: 0.7524 - val_loss: 0.0191 - val_acc: 0.7524
Epoch 475/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0190 - acc: 0.7523 - val_loss: 0.0192 - val_acc: 0.7524
Epoch 476/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0190 - acc: 0.7526 - val_loss: 0.0191 - val_acc: 0.7543
Epoch 477/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0190 - acc: 0.7547 - val_loss: 0.0190 - val_acc: 0.7543
Epoch 478/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0189 - acc: 0.7538 - val_loss: 0.0190 - val_acc: 0.7537
Epoch 479/2000
9700/9700 [=======================

...

9700/9700 [==============================] - 0s 5us/step - loss: 0.0063 - acc: 0.8994 - val_loss: 0.0062 - val_acc: 0.9028
Epoch 1712/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0062 - acc: 0.8986 - val_loss: 0.0062 - val_acc: 0.9021
Epoch 1713/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0062 - acc: 0.8984 - val_loss: 0.0062 - val_acc: 0.9030
Epoch 1714/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0062 - acc: 0.8985 - val_loss: 0.0063 - val_acc: 0.9031
Epoch 1715/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0063 - acc: 0.8989 - val_loss: 0.0061 - val_acc: 0.9031
Epoch 1716/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0064 - acc: 0.8989 - val_loss: 0.0062 - val_acc: 0.9025
Epoch 1717/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0063 - acc: 0.8977 - val_loss: 0.0064 - val_acc: 0.9031
Epoch 1718/2000
9700/9700 [================

9700/9700 [==============================] - 0s 5us/step - loss: 0.0061 - acc: 0.9008 - val_loss: 0.0062 - val_acc: 0.9051
Epoch 1771/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0060 - acc: 0.9006 - val_loss: 0.0060 - val_acc: 0.9046
Epoch 1772/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0061 - acc: 0.9000 - val_loss: 0.0060 - val_acc: 0.9044
Epoch 1773/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0060 - acc: 0.8994 - val_loss: 0.0061 - val_acc: 0.9052
Epoch 1774/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0060 - acc: 0.9013 - val_loss: 0.0061 - val_acc: 0.9055
Epoch 1775/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0060 - acc: 0.8999 - val_loss: 0.0060 - val_acc: 0.9039
Epoch 1776/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0061 - acc: 0.8989 - val_loss: 0.0060 - val_acc: 0.9049
Epoch 1777/2000
9700/9700 [================

9700/9700 [==============================] - 0s 5us/step - loss: 0.0059 - acc: 0.9021 - val_loss: 0.0059 - val_acc: 0.9071
Epoch 1830/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0059 - acc: 0.9024 - val_loss: 0.0059 - val_acc: 0.9066
Epoch 1831/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0059 - acc: 0.9016 - val_loss: 0.0059 - val_acc: 0.9071
Epoch 1832/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0059 - acc: 0.9026 - val_loss: 0.0059 - val_acc: 0.9067
Epoch 1833/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0060 - acc: 0.9021 - val_loss: 0.0059 - val_acc: 0.9065
Epoch 1834/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0060 - acc: 0.9024 - val_loss: 0.0059 - val_acc: 0.9072
Epoch 1835/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0059 - acc: 0.9037 - val_loss: 0.0060 - val_acc: 0.9060
Epoch 1836/2000
9700/9700 [================

9700/9700 [==============================] - 0s 5us/step - loss: 0.0057 - acc: 0.9030 - val_loss: 0.0057 - val_acc: 0.9080
Epoch 1889/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0058 - acc: 0.9044 - val_loss: 0.0057 - val_acc: 0.9085
Epoch 1890/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0057 - acc: 0.9046 - val_loss: 0.0058 - val_acc: 0.9085
Epoch 1891/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0058 - acc: 0.9042 - val_loss: 0.0058 - val_acc: 0.9093
Epoch 1892/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0057 - acc: 0.9043 - val_loss: 0.0057 - val_acc: 0.9078
Epoch 1893/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0058 - acc: 0.9038 - val_loss: 0.0057 - val_acc: 0.9086
Epoch 1894/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0058 - acc: 0.9046 - val_loss: 0.0058 - val_acc: 0.9094
Epoch 1895/2000
9700/9700 [================

9700/9700 [==============================] - 0s 5us/step - loss: 0.0057 - acc: 0.9062 - val_loss: 0.0056 - val_acc: 0.9101
Epoch 1948/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0056 - acc: 0.9064 - val_loss: 0.0056 - val_acc: 0.9102
Epoch 1949/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0056 - acc: 0.9060 - val_loss: 0.0056 - val_acc: 0.9100
Epoch 1950/2000
9700/9700 [==============================] - 0s 4us/step - loss: 0.0056 - acc: 0.9062 - val_loss: 0.0056 - val_acc: 0.9100
Epoch 1951/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0056 - acc: 0.9067 - val_loss: 0.0056 - val_acc: 0.9102
Epoch 1952/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0056 - acc: 0.9066 - val_loss: 0.0056 - val_acc: 0.9099
Epoch 1953/2000
9700/9700 [==============================] - 0s 5us/step - loss: 0.0056 - acc: 0.9061 - val_loss: 0.0056 - val_acc: 0.9099
Epoch 1954/2000
9700/9700 [================

After training for 2000 epochs, let's check the model performance with our reserved evaluation test data.

In [7]:
model.evaluate(X[-3000:],Y[-3000:])

3000/3000 [==============================] - 0s 11us/step


[0.006693016894413935, 0.905666666507721]